<a href="https://colab.research.google.com/github/PavanDaniele/drone-person-detection/blob/main/model_Training_and_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up: mount drive + import libraries

**Important Information:** We need to activate the GPU on Colab (_Runtime --> Change runtime type_). \
Every time you start a new session (or reopen the notebook after a few hours) check that the GPU is still active. If we are not using the GPU it can take up to tens of hours to train the models. \
_GPU T4 is the best choice._

In [24]:
# Run this Every time you start a new session
from google.colab import drive
drive.mount('/content/drive') # to mount google drive (to see/access it)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!pip install ultralytics # Installation of Ultralytics for YOLO models

In [26]:
from ultralytics import YOLO # Import of Ultralytics for YOLO models
import shutil
import os
from PIL import Image

# General Explanation

### Backbone

In Computer Vision, a _Backbone_ is the part of a convolutional neural network responsible for extracting the main features from an image. \
It serves as the shared base upon which subsequent modules are built (such as heads for classification, object detection, segmentation, etc.).

\
Each backbone has been pre-trained on specific datasets (e.g., ImageNet) using particular preprocessing steps, input dimensions, normalization, and augmentation techniques, which should ideally be replicated during fine-tuning to maintain compatibility and achieve optimal performance.

### Data Loader

To train a deep learning model, it is essential to properly handle data loading and preparation. This is the task of the _Data Loader_, a component responsible for:
- Loading images and their corresponding annotations (e.g., .txt or .json) from the dataset.
- Applying preprocessing operations, such as resizing, normalization, data augmentation, etc.
- Organizing data into batches to feed the model during training.

\
Considering the limited resources of my development environment, at first I decided to perform the image and annotation resizing in a separate phase (prior to training), in order to:
- Reduce the workload of the data loader during fine-tuning;
- Increase data loading and training speed;
- Ensure consistency between images and annotations.

But due to the different type of scaling technique, I want to try to fine-tuning the model without any pre-scaling.

The other transformations, instead, are handled by the model-specific data loader, since each model uses different preprocessing and normalization techniques. \
Moreover, some models require specific transformations to achieve optimal performance, and the libraries that provide the models (e.g., Ultralytics for YOLO, torchvision for EfficientDet/SSD) already implement loaders that are properly configured and optimized.

### Normalization

Image normalization consists in scaling pixel values from the range [0, 255] to a more suitable interval (e.g., [0, 1] or [-1, 1]), often based on the mean and standard deviation of the pre-training dataset, with the goal of:
- Avoiding overly large values in the tensors;
- Making the model more stable during training;
- Speeding up convergence.

\
Normalization helps maintain a consistent pixel range and distribution, which is essential for pre-trained models.

### Data Augmentations

Data augmentation consists of random transformations (e.g., rotations, flips, crops, brightness changes, etc.) applied during training. Their purpose is to:
- Simulate new visual conditions;
- Increase dataset variety;
- Reduce overfitting by improving the model’s ability to generalize.

\
In practice, the semantic content of the image doesn't change (e.g., a person remains a person), but its visual appearance is altered to help the model "learn better."


My goal is to evaluate the real-world performance of each model in its ideal scenario, in order to select the most suitable one for deployment on the Jetson Nano. \
For this reason, each model is trained using its native augmentations, meaning the ones that were designed and optimized as part of its original architecture. \
It wouldn’t make sense to disable them or enforce a uniform setup across models, because what we want to observe is the maximum potential of each model, working in the way it was designed to perform best.

# Fine-Tuning Model

### YOLOv8n

First of all we need to save the dataset locally:


In [27]:
src = '/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n'
dst = '/content/AERALIS_YOLOv8n_local'  # is now on the local VM, NOT on drive

# If the destination folder already exists, I delete it
if os.path.exists(dst):
  shutil.rmtree(dst)

# Recursive copy of ENTIRE folder (and subfolders)
shutil.copytree(src, dst)
print("Copy completed:", os.path.exists(dst))

Copy completed: True


Let's check the total free space:

In [28]:
!df -h / # It shows the total, used and free space on the root (/) of the Colab VM.

# Avail column: space still available for your files.

Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   56G   58G  49% /


In [29]:
# Show space used by your local folder
!du -sh /content/AERALIS_YOLOv8n_local

6.6G	/content/AERALIS_YOLOv8n_local


In [30]:
# Show space occupied by various folders in /content/.
!du -h --max-depth=1 /content/

140K	/content/.config
du: cannot access '/content/drive/.Encrypted/.shortcut-targets-by-id/1LQbD7p_iS5KLqGNdfrYEvsAx0i_bgB0h/projectUPV': No such file or directory
67G	/content/drive
6.6G	/content/AERALIS_YOLOv8n_local
31M	/content/runs_finetune
55M	/content/sample_data
74G	/content/


We want to create the data.yaml file, which YOLO uses to know:
- the path to the training, validation, and test images
- the number of classes (nc)
- the names of the classes (names)

\
This file is used by YOLO to locate the images and their annotations.

In [31]:
# YAML dataset (edit routes)
data_yaml = """
train: /content/AERALIS_YOLOv8n_local/train/images
val:   /content/AERALIS_YOLOv8n_local/val/images
test:  /content/AERALIS_YOLOv8n_local/test/images

nc: 1
names: ['person']
"""
open('data.yaml', 'w').write(data_yaml)

176

Perfect, we have correctly written the data.yaml file for the AERALIS_YOLOv8n dataset. Let's continue with the loading of the model:

In [32]:
# Upload the pre-trained model we want to use as a starting point

model_YOLOv8n = YOLO('yolov8n.pt') # it is the model that will be fine-tuned on the custom dataset

We have now downloaded the pre-trained model from the official Ultralytics repository.

The **Batch size** is the number of images processed simultaneously in each training step. With 4-8GB of RAM, a batch size of 8 (or even less) is recommended, so we'll start with that value and reduce it if necessary.

**Early Stopping** is a technique that automatically stops the training process if the model stops improving after a certain number of epochs. This helps prevent overfitting and saves time.

**Workers** are the parallel processes used to load and preprocess data while the model is training. However, due to our limited resources, we’ll start with 2 workers, and if data loading errors occur, we'll reduce this number to 1 or even 0.


In [33]:
# To see the available GPU
import torch
print(torch.cuda.is_available()) # True = you have GPU --> if False then use device='cpu'
print(torch.cuda.device_count()) # Name of GPU

# If True and at least 1, you can use device=0.
# If you don't have GPU: use device='cpu' (much slower).
# Locally (not Colab): check with nvidia-smi from terminal.

True
1


Now that we have confirmation that the GPU is active we can train the model:

In [35]:
# Fine‑tuning
results_YOLOv8n = model_YOLOv8n.train(
  data='data.yaml', # use the newly created yaml file
  epochs=100, # Maximum number of training epochs
  imgsz=640, # Image input size (recommended for YOLO).
  batch=16,  # Batch size
  patience=20, # Early stopping if the metrics do not improve for 20 epochs
  workers=2, # Number of workers for the dataloader
  device=0, # Use GPU 0 (or put 'cpu' if you don't have GPU)
  # device='cpu',
  project='runs_finetune', # Folder where it will save the results of the experiments (the folder will be created automatically)
  name='person_yolov8n' # Subfolder/name specific to our experiment
)

# results -->  will contain metrics, logs, and the path of the best weights found during the training

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=person_yolov8n4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12.0, pretrained=True, prof

train: Scanning /content/AERALIS_YOLOv8n_local/train/labels.cache... 2395 images, 388 backgrounds, 0 corrupt: 100%|██████████| 2395/2395 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 901.0±20.7 MB/s, size: 2606.8 KB)


val: Scanning /content/AERALIS_YOLOv8n_local/val/labels.cache... 515 images, 75 backgrounds, 0 corrupt: 100%|██████████| 515/515 [00:00<?, ?it/s]


...

    100/100      2.48G      1.154     0.5461     0.8617         15        640: 100%|██████████| 150/150 [01:04<00:00,  2.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  2.86it/s]

                   all        515       1258      0.896      0.769      0.857      0.481



100 epochs completed in 2.009 hours.
Optimizer stripped from runs_finetune/person_yolov8n4/weights/last.pt, 6.2MB
Optimizer stripped from runs_finetune/person_yolov8n4/weights/best.pt, 6.2MB

Validating runs_finetune/person_yolov8n4/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  2.87it/s]


                   all        515       1258      0.905      0.771      0.861      0.482
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs_finetune/person_yolov8n4


We now want to evaluate the trained model using the Test set defined in data.yaml. \
YOLO does not compute standard accuracy, because in object detection True Negatives (TN) are not counted. Therefore, traditional accuracy is not applicable or useful.


So, we will compute:
- **Precision**: how correct your detected positives are
- **Recall**: how many of the real objects you detected
- **mAP50**: mean Average Precision with IoU ≥ 0.5 (how accurate the predictions are)
- **mAP50-95**: average over various IoU thresholds, a more strict metric
- **F1_score**: combination of precision and recall (you can compute it as: 2 * (P * R) / (P + R))

In [36]:
# Evaluates the trained model using the TEST SET defined in data.yaml

metrics_YOLOv8n = model_YOLOv8n.val(data='data.yaml', split='test') # returns accuracy metrics (e.g., mAP, precision, recall, etc.) on the test set
print("Test metrics:", metrics_YOLOv8n)

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 80.0±11.5 MB/s, size: 1605.8 KB)


val: Scanning /content/AERALIS_YOLOv8n_local/test/labels... 516 images, 93 backgrounds, 0 corrupt: 100%|██████████| 516/516 [00:01<00:00, 505.99it/s]

val: New cache created: /content/AERALIS_YOLOv8n_local/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:15<00:00,  2.10it/s]


                   all        516       1271      0.905      0.829      0.904      0.506
Speed: 0.3ms preprocess, 1.0ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs_finetune/person_yolov8n42
Test metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d498713b390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031

Confidence (confidence) is the probability estimated by the model that a detected object is actually real (i.e., not a false positive).

In [41]:
#  Inference (practical use of the fine-tuned model)

# Load the best weights found
best_YOLOv8n = '/content/runs_finetune/person_yolov8n2/weights/best.pt'
model_inf_YOLOv8n = YOLO(best_YOLOv8n) # Creates a new model instance by loading the best weights

# Performs inference on one or more images, or on a video, by specifying the path in the source parameter.
# conf=0.25 → Confidence threshold for considering a detection valid.
preds_YOLOv8n = model_inf_YOLOv8n.predict(
  # source='/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/test/images',
  source='/content/AERALIS_YOLOv8n_local/test/images',
  conf=0.25,
  verbose=False, # disable on-screen printing
  save=True
)

# Too many images to show!
# for result in preds_YOLOv8n:
#    result.show() # displays the predictions (eventually you can also save them)

Results saved to runs/detect/predict


In order to use it on the Jetson Nano we now want to download the model, so:

In [42]:
# Create folders on Google Drive (first time only):
!mkdir -p /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/results/annotated
!mkdir -p /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/weights

In [44]:
# Copy annotated images:
!cp -r /content/runs/detect/predict/* /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/results/annotated/

# Copy the best weights (best.pt) after training:
!cp /content/runs_finetune/person_yolov8n2/weights/best.pt /content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/weights/

In [ ]:
# To download it directly to the computer:
from google.colab import files
files.download('/content/runs_finetune/person_yolov8n/weights/best.pt')

When we have the .pt file on the PC, we could:
- Upload it to the Jetson Nano,
- Convert it to ONNX/TensorRT if you need it for optimization,
- Use it with the PyTorch/Ultralytics version on any computer.

Weight access code

In [ ]:
# Path of saved weights:
weights_path_YOLOv8n = '/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/weights/best.pt'
model_YOLOv8n = YOLO(weights_path_YOLOv8n)

Access code for annotated images

In [ ]:
# Directory of annotated images
annotated_dir = '/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv8n/results/annotated'

# Get all the .jpg files in the directory
annotated_imgs_YOLOv8n = [
    os.path.join(annotated_dir, f)
    for f in os.listdir(annotated_dir)
    if f.lower().endswith('.jpg')
]

# View the first 5 annotated images
for img_path in annotated_imgs_YOLOv8n[:5]:
  img = Image.open(img_path)
  img.show()

### YOLOv11n

Ora eseguiamo lo stesso procedimento per YOLOv11n:

In [ ]:
src = '/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv11n'
dst = '/content/AERALIS_YOLOv11n_local'  # is now on the local VM, NOT on drive

# If the destination folder already exists, I delete it
if os.path.exists(dst):
  shutil.rmtree(dst)

# Recursive copy of ENTIRE folder (and subfolders)
shutil.copytree(src, dst)
print("Copy completed:", os.path.exists(dst))

In [ ]:
!df -h / # It shows the total, used and free space on the root (/) of the Colab VM.

# Avail column: space still available for your files.

In [ ]:
# Show space used by your local folder
!du -sh /content/AERALIS_YOLOv11n_local

In [ ]:
# Show space occupied by various folders in /content/.
!du -h --max-depth=1 /content/

In [ ]:
# YAML dataset (edit routes)
data_yaml = """
train: /content/AERALIS_YOLOv11n_local/train/images
val:   /content/AERALIS_YOLOv11n_local/val/images
test:  /content/AERALIS_YOLOv11n_local/test/images

nc: 1
names: ['person']
"""
open('data.yaml', 'w').write(data_yaml)

In [ ]:
# Carica il modello pre-addestrato che vogliamo usare come punto di partenza

model_YOLOv11n = YOLO('yolov11n.pt') # è il modello che verrà fine-tunato sul dataset custom

In [ ]:
# Fine‑tuning
results_YOLOv11n = model_YOLOv11n.train(
  data='data.yaml',
  epochs=100,
  imgsz=640,
  batch=16,
  patience=20,
  workers=2,
  device=0,
  project='runs_finetune',
  name='person_yolov11n'
)

In [ ]:
# Evaluates the trained model using the TEST SET defined in data.yaml

metrics_YOLOv11n = model_YOLOv11n.val(data='data.yaml', split='test') # returns accuracy metrics (e.g., mAP, precision, recall, etc.) on the test set
print("Test metrics:", metrics_YOLOv11n)

In [ ]:
#  Inference (practical use of the fine-tuned model)

# I load the best weights found
best_YOLOv11n = results_YOLOv11n.best or f'runs_finetune/person_yolov11n/weights/best.pt' # or '...': is a manual backup in case results.best does not exist

model_inf_YOLOv11n = YOLO(best_YOLOv11n) # Creates a new model instance by loading the best weights

# Performs inference on one or more images, or on a video, by specifying the path in the source parameter.
# conf=0.25 → Confidence threshold for considering a detection valid.
preds_YOLOv11n = model_inf_YOLOv11n.predict(
  # source='/content/drive/MyDrive/projectUPV/datasets/AERALIS_YOLOv11n/test/images',
  source='/content/AERALIS_YOLOv11n_local/test/images',
  conf=0.25
)
preds_YOLOv11n.show() # displays the predictions (eventually you can also save them)

### EfficientDet D0

Iniziamo ora ad allenare il modello EfficientDet. Utilizzeremo la libreria _effdet_ di _Ross Wightman_, che semplifica tutto il processo:

In [ ]:
!pip install effdet
!pip install pycocotools


In [ ]:
import torch
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
from torch.utils.data import DataLoader
from torchvision import transforms as T
from effdet.data.datasets import CocoDataset
from effdet.data.loader import create_loader
import os

# MODEL CONFIG
model_name = 'tf_efficientdet_d0'  # Cambia in 'd1', 'd2' se vuoi
num_classes = 2  # 1 classe + background
image_size = 512
batch_size = 8
epochs = 50
lr = 1e-4

# EfficientDet config
config = get_efficientdet_config(model_name)
config.image_size = image_size
config.num_classes = num_classes
config.norm_kwargs = dict(eps=1e-3, momentum=0.01)

# Model
net = EfficientDet(config, pretrained_backbone=True)
net.class_net = HeadNet(config, num_outputs=num_classes)
model = DetBenchTrain(net, config)
model = model.cuda()

# Dataset (COCO format)
train_dataset = CocoDataset(
    '/content/dataset/train/images',
    '/content/dataset/annotations/instances_train.json',
    transform=T.ToTensor()
)

val_dataset = CocoDataset(
    '/content/dataset/val/images',
    '/content/dataset/annotations/instances_val.json',
    transform=T.ToTensor()
)

# Dataloader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, targets in train_loader:
        images = torch.stack(images).cuda()
        targets = [{k: v.cuda() for k, v in t.items()} for t in targets]

        loss, _ = model(images, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"[Epoch {epoch+1}/{epochs}] Training Loss: {total_loss:.4f}")

# Save model
torch.save(model.state_dict(), f'efficientdet_d0_custom.pt')  # Cambia nome in base alla versione


In [ ]:
!pip install effdet timm

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchvision import transforms
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchPredict

# 1️⃣ Trasformazioni (ToTensor + ImageNet-normalizzazione)
_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 2️⃣ Dataset COCO train/val
train_ds = CocoDetection(root="/content/datasets/train/images",
                         annFile="/content/datasets/annotations_train.json",
                         transform=_transform)
val_ds = CocoDetection(root="/content/datasets/val/images",
                       annFile="/content/datasets/annotations_val.json",
                       transform=_transform)

def train_efficientdet(model_name, image_size, epochs=20, bs=8, lr=1e-4):
    # 3️⃣ Configura e istanzia modello
    config = get_efficientdet_config(f'tf_efficientdet_{model_name}')
    config.num_classes = 1
    config.image_size = image_size
    model = EfficientDet(config, pretrained_backbone=True)
    model = DetBenchTrain(model, config).to(device)

    loader_train = DataLoader(train_ds, batch_size=bs, shuffle=True,
                              collate_fn=lambda x: tuple(zip(*x)))
    loader_val = DataLoader(val_ds, batch_size=bs, shuffle=False,
                            collate_fn=lambda x: tuple(zip(*x)))

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    # 4️⃣ Ciclo training + validation
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for imgs, targets in loader_train:
            imgs = torch.stack(imgs).to(device)
            loss_dict = model(imgs, targets)
            loss = sum(loss_dict.values())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss/len(loader_train)

        model.eval()
        val_loss = sum(sum(model(torch.stack(imgs).to(device), tg).values()).item()
                        for imgs,tg in loader_val) / len(loader_val)

        print(f"[{model_name} | epoch {epoch}/{epochs}] train_loss={avg_loss:.4f}, val_loss={val_loss:.4f}")

    return model, config

# Imposta device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ✅ Fine-tuning per D0, D1, D2
models = {}
for name, size in [('d0',512), ('d1',640), ('d2',768)]:
    models[name] = train_efficientdet(name, image_size=size)



In [ ]:
def run_inference(model, config, img_path, conf_thr=0.25):
    model_eval = DetBenchPredict(model.model, config).to(device).eval()
    from PIL import Image
    import numpy as np

    img = Image.open(img_path).convert('RGB')
    inp = _transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model_eval(inp)[0]
    # Filtra in base alla soglia
    keep = out['scores'] > conf_thr
    boxes = out['boxes'][keep].cpu().numpy()
    scores = out['scores'][keep].cpu().numpy()
    return boxes, scores

# Esempio
boxes, scores = run_inference(models['d0'][0], models['d0'][1], "/content/datasets/val/images/sample1.jpg")
print("Boxes:", boxes.shape, "Scores:", scores)


In [ ]:
# Installa effdet (Ross Wightman)
!pip install effdet

# Per dataset COCO in PyTorch
!pip install pycocotools


In [ ]:
import os
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchEval
from effdet.efficientdet import HeadNet
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchvision import transforms

# Parametri
DATASET_DIR = '/content/AERALIS_EfficientDet_D0_local'
NUM_CLASSES = 1  # solo 'person' (non mettere background!)
BATCH_SIZE = 8
EPOCHS = 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Funzione per creare i DataLoader COCO ---
def get_coco_loader(img_dir, ann_path, batch_size, shuffle):
    dataset = CocoDetection(
        img_dir, ann_path,
        transforms=transforms.Compose([
            transforms.ToTensor()
        ])
    )
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))

# --- Scegli la variante EfficientDet (D0, D1, D2) ---
# Sostituisci 'tf_efficientdet_d0' con 'tf_efficientdet_d1' o 'tf_efficientdet_d2' se vuoi
variant = 'tf_efficientdet_d0'
config = get_efficientdet_config(variant)
config.num_classes = NUM_CLASSES
config.image_size = 512  # D0=512, D1=640, D2=768

net = EfficientDet(config, pretrained_backbone=True)
net.class_net = HeadNet(config, num_outputs=NUM_CLASSES)
model = DetBenchTrain(net, config).to(DEVICE)

# --- Dataloader ---
train_loader = get_coco_loader(
    os.path.join(DATASET_DIR, 'train/images'),
    os.path.join(DATASET_DIR, 'annotations_train.json'),
    BATCH_SIZE, shuffle=True
)
val_loader = get_coco_loader(
    os.path.join(DATASET_DIR, 'val/images'),
    os.path.join(DATASET_DIR, 'annotations_val.json'),
    BATCH_SIZE, shuffle=False
)

# --- Ottimizzatore ---
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# --- Training Loop ---
for epoch in range(EPOCHS):
    model.train()
    for images, targets in train_loader:
        images = [img.to(DEVICE) for img in images]
        boxes = [torch.tensor([obj['bbox'] for obj in t], dtype=torch.float32).to(DEVICE) for t in targets]
        labels = [torch.tensor([obj['category_id'] for obj in t], dtype=torch.int64).to(DEVICE) for t in targets]
        target_dicts = [{'bbox': b, 'cls': l} for b, l in zip(boxes, labels)]
        loss_dict = model(images, target_dicts)
        loss = loss_dict['loss']
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {loss.item():.4f}")

    # Salva i pesi migliori
    torch.save(model.state_dict(), f'effdet_{variant}_epoch{epoch+1}.pth')

# Salva i pesi finali
torch.save(model.state_dict(), f'effdet_{variant}_final.pth')
print("Fine-tuning completato.")


In [ ]:
from effdet import DetBenchEval

# Carica il modello in modalità evaluation
net = EfficientDet(config, pretrained_backbone=False)
net.class_net = HeadNet(config, num_outputs=NUM_CLASSES)
model = DetBenchEval(net, config).to(DEVICE)
model.load_state_dict(torch.load(f'effdet_{variant}_final.pth'))
model.eval()

# Esempio: predizione su un batch di immagini validation
with torch.no_grad():
    for images, targets in val_loader:
        images = [img.to(DEVICE) for img in images]
        preds = model(images)
        print("Predictions for a batch:", preds)
        break  # rimuovi per valutare su tutto il validation set

# Per valutazione COCO (mAP, precision, recall), puoi usare pycocotools e funzioni custom,
# oppure vedere la sezione "evaluation" della repo (alcuni script di esempio sono inclusi).


### EfficientDet D1

### EfficientDet D2

### MobileNetV2 + SSDLite

### MobileNetV3 + SSDLite

# Normalization and Data Augmentation

Per i modelli leggeri ottimizzati come quelli per Jetson Nano, la normalizzazione delle immagini è quasi sempre richiesta prima di passarle al modello.

I modelli in PyTorch lavorano SOLO con tensori, NON con immagini PIL o array NumPy.
- ToTensor() converte un’immagine (PIL o NumPy) in un tensore PyTorch di tipo float32, formato [C, H, W] (canale, altezza, larghezza).

- Inoltre, scala i valori dei pixel da [0,255] a [0,1] automaticamente.

La normalizzazione (Normalize) funziona SOLO su tensori.
La funzione Normalize(mean, std) richiede input già in formato tensore (float) e applica lo shift/scala canale per canale.

Se provi a normalizzare un’immagine PIL o un NumPy array direttamente, ottieni errore o comportamenti inattesi.

Quindi: la sequenza è SEMPRE
(Opzionale) Resize

ToTensor()   →  Converte e scala [0,255] in [0,1]

Normalize()  →  Normalizza ogni canale secondo mean/std richiesto dal modello


\
 In sintesi:
ToTensor è indispensabile, non è solo per PyTorch, ma anche perché la normalizzazione funziona SOLO su tensori, non su immagini raw!

La normalizzazione NON sostituisce ToTensor: lavora sopra i dati già convertiti.

La sequenza ToTensor() + normalizzazione è quasi sempre necessaria, ma i dettagli della normalizzazione (mean, std, range pixel) possono cambiare in base al modello.
Vediamo la situazione per i tuoi modelli:
-


# Fine-Tuning Models (Alexia)

Analizzo il codice di Alexia per avere uno spunto:

1. test_comptage_img.py
Scopo:
Carica un modello YOLO addestrato e conta quanti oggetti della classe 0 (qui chiamati "oiseaux" = uccelli, ma tu potresti adattare a "persone") vengono rilevati in una singola immagine.

In [ ]:
from ultralytics import YOLO # Importa la libreria Ultralytics YOLO

# Chargement du modèle entraîné
model = YOLO("../runs/detect/train4/weights/best.pt") # Carica il modello YOLO addestrato dal file best.pt (specificare il percorso giusto)

# Prédiction sur une image
results = model.predict("test4.jpeg") # Esegue la predizione sull'immagine "test4.jpeg"

# Compte des oiseaux (classe 0)
bird_count = sum(1 for cls in results[0].boxes.cls if int(cls) == 0) # Conta quante bounding box appartengono alla classe 0

print(f"Nombre d'oiseaux : {bird_count}") # Stampa il numero di oggetti (classe 0) rilevati


Nota:

Puoi cambiare "classe 0" con "persona" se il tuo modello rileva persone come classe 0.

2. test_comptage_video.py
Scopo:
Carica un modello YOLO addestrato, effettua il tracking (con ByteTrack) e conta quanti oggetti della classe 0 ("oiseaux") entrano in un rettangolo centrale all'interno di un video.
Annota la video con bounding box, ID, conta corrente e totale degli oggetti unici che sono entrati nel rettangolo.

In [ ]:
from ultralytics import YOLO                # Importa YOLO da Ultralytics
import cv2                                  # Importa OpenCV per gestione video e immagini
import os                                   # Importa os (qui non usato, ma spesso per path)

# Charger le modèle
model = YOLO("/Users/alexiagaido--amoros/Desktop/UPV-test/entrainement_serveur/runs/detect/train9/weights/best.pt")
# Carica il modello YOLO addestrato (specifica percorso)

# Chemin de la vidéo
video_path = "img_video/video_test_1.mp4"   # Path della video da analizzare
output_path = "output_video.mp4"            # Path della video annotata in output

# Distance des bords pour le rectangle de contact (en pixels)
border_distance = 50                        # Margine dai bordi (pixels) per il rettangolo centrale

# Ouvrir la vidéo
cap = cv2.VideoCapture(video_path)          # Apre la video
if not cap.isOpened():
    print("Erreur : Impossible d'ouvrir la vidéo")   # Se non apre la video, errore
    exit()

# Obtenir les propriétés de la vidéo
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))       # Ottiene larghezza frame
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))     # Ottiene altezza frame
fps = int(cap.get(cv2.CAP_PROP_FPS))                # Ottiene fps

# Définir les coordonnées du rectangle de contact
rect_x1 = border_distance                           # Coordinate x1 del rettangolo
rect_y1 = border_distance                           # Coordinate y1
rect_x2 = width - border_distance                   # Coordinate x2
rect_y2 = height - border_distance                  # Coordinate y2

# Configurer la sortie vidéo
fourcc = cv2.VideoWriter_fourcc(*"mp4v")            # Codec video per output
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))  # Oggetto per scrivere la video annotata

# Ensemble pour stocker les IDs uniques des oiseaux dans le rectangle
unique_bird_ids = set()                             # Insieme per salvare gli ID unici degli oggetti che sono passati nel rettangolo

while cap.isOpened():
    ret, frame = cap.read()                         # Leggi un frame
    if not ret:
        break

    # Effectuer l'inférence avec suivi
    results = model.track(frame, conf=0.5, tracker="bytetrack.yaml", persist=True)
    # Fa inferenza + tracking, usa ByteTrack, restituisce risultati con ID di tracking

    # Compter les oiseaux dans cette frame
    bird_count = 0
    if results[0].boxes.id is not None:             # Se ci sono ID di tracking
        for box, box_id in zip(results[0].boxes, results[0].boxes.id):   # Scorri bounding box e relativi ID
            # Vérifier si le centre de la bounding box est dans le rectangle
            x_center = (box.xyxy[0][0] + box.xyxy[0][2]) / 2            # Calcola centro x
            y_center = (box.xyxy[0][1] + box.xyxy[0][3]) / 2            # Calcola centro y
            if rect_x1 < x_center < rect_x2 and rect_y1 < y_center < rect_y2:   # Se centro box dentro rettangolo centrale
                unique_bird_ids.add(box_id.item())                      # Aggiungi ID a set (oggetti unici che sono passati)
                bird_count += 1                                         # Conta per questa frame

    # Annoter l'image avec les détections et IDs
    annotated_frame = results[0].plot()              # Disegna box e ID sul frame

    # Dessiner le rectangle de contact
    cv2.rectangle(
        annotated_frame,
        (rect_x1, rect_y1),
        (rect_x2, rect_y2),
        (255, 0, 0),  # Blu
        2,            # Spessore linea
    )

    # Afficher le nombre d'oiseaux dans cette frame et le total unique
    cv2.putText(
        annotated_frame,
        f"Oiseaux dans cette frame : {bird_count}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),  # Verde
        2,
    )
    cv2.putText(
        annotated_frame,
        f"Oiseaux uniques : {len(unique_bird_ids)}",
        (10, 60),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2,
    )

    # Écrire l'image annotée dans la vidéo de sortie
    out.write(annotated_frame)

    # Afficher l'image en temps réel
    cv2.imshow("YOLO Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):  # Premere 'q' per uscire
        break

# Afficher le total des oiseaux uniques détectés
print(f"Nombre total d'oiseaux uniques détectés dans la vidéo : {len(unique_bird_ids)}")

# Libérer les ressources
cap.release()
out.release()
cv2.destroyAllWindows()

Considerazioni tecniche
Classe 0: Il codice è pensato per oggetti "oiseaux" (uccelli) = classe 0. Se tu hai persone come classe 0, funziona identico.

Tracking (ByteTrack): Permette di assegnare un ID a ogni oggetto/persona che attraversa l’area, così da contarli solo una volta anche se si fermano/muovono nella scena.

Rettangolo di interesse: Conta solo gli oggetti il cui centro entra in una zona centrale, utile ad esempio per contare solo chi passa in una certa area (adattabile per ingressi, uscite, ecc).

Salvataggio video annotato: Il risultato è un video con box, ID e conteggi stampati sopra.